In [1]:
import pathlib
import pandas as pd
import numpy as np
from itertools import combinations
import os
from tqdm import tqdm
import requests
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

data_path = data_path = pathlib.Path().resolve().parent.parent / "rn_home" / "data"
rlpolk_data_path = data_path / "rlpolk_data"
vin_matching_path = data_path / "vin_matching"

# Define required functions

In [2]:
def try_divide(a,b, num =2):
    try:
        return round(a/b,num)
    except:
        return np.NaN

In [3]:
def expand_df(df, oldyear, newyear):
    df_cut = df.loc[df["year"]==oldyear,:]
    df_cut["year"] = newyear
    df_out = pd.concat([df, df_cut])
    return df_cut

# Import data

In [31]:
rlp_raw = pd.read_csv(rlpolk_data_path / "rlpolk_data_matched.csv", index_col = [0])
evs_raw = rlp_raw[rlp_raw["Fuel Type - Primary"]=="Electric"]

In [32]:
rlp = rlp_raw.copy(deep=True)

# Prepare block group allocations

In [33]:
mun_evs = pd.read_csv(data_path / "municipal_evs_zips_blockgroups" / "evs_geocoded_zip_blockgroup.csv")

In [34]:
cols_keep = ['Status', 'Region', 'Postal', 'PostalExt', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID']
mun_evs = mun_evs.loc[:,cols_keep]
mun_evs = mun_evs.loc[(mun_evs["Region"]=="Connecticut") & (mun_evs["Status"]=="M"), :]
mun_evs["Postal"] = mun_evs["Postal"].astype(int)

In [35]:
mun_evs_zipgroups = mun_evs.groupby(["Postal", "GEOID"]).count().reset_index()[["Postal", "GEOID", "Status"]].rename(columns={"Status":"Count"})

In [36]:
mun_evs_zipgroups["pct_within_zip"] = mun_evs_zipgroups["Count"] / mun_evs_zipgroups.groupby("Postal")["Count"].transform('sum')

In [37]:
mun_evs_zipgroups[["Postal", "GEOID", "pct_within_zip"]]

,Postal,GEOID,pct_within_zip
0,6001,9.003460e+10,0.009259
1,6001,9.003462e+10,0.003086
2,6001,9.003462e+10,0.104938
3,6001,9.003462e+10,0.064815
4,6001,9.003462e+10,0.037037
...,...,...,...
2441,6907,9.001021e+10,0.113208
2442,6907,9.001021e+10,0.245283
2443,6907,9.001021e+10,0.094340
2444,6907,9.001021e+10,0.037736


In [38]:
rlp_zips = evs_raw["ZIP_CODE"].unique().tolist()

# Quality check - Price Data

In [39]:
# Check how many vehicles in the dataset have a price attached to them
price_mask = evs_raw["TRANSACTION_PRICE"].notna()
vehs_price = evs_raw[price_mask]["VEH_COUNT"].sum()
vehs_noprice = evs_raw[~price_mask]["VEH_COUNT"].sum()
print(f"{vehs_price/(vehs_noprice+vehs_price) * 100:0.2f}% of vehicles have a price")

77.11% of vehicles have a price


In [40]:
# Count vehicles with and without prices
rlp_price_mask = rlp_raw["TRANSACTION_PRICE"].notna()
rlp.loc[rlp_price_mask, "vehs_with_price"] = rlp.loc[rlp_price_mask, "VEH_COUNT"]
rlp.loc[~rlp_price_mask, "vehs_with_price"] = 0
rlp.loc[~rlp_price_mask, "vehs_without_price"] = rlp.loc[~rlp_price_mask, "VEH_COUNT"]
rlp.loc[rlp_price_mask, "vehs_without_price"] = 0

# Group Data

We need our observations to be by:
* Type (electric)
* Zip code
* Year & month
Along with the average transaction price

In [41]:
# Prepare for grouping
rlp_for_group = rlp[["ZIP_CODE", "year", "month", "Fuel Type - Primary", "VEH_COUNT", "vehs_with_price", "vehs_without_price", "TRANSACTION_PRICE"]]

# Group to get counts and averages
rlp_grouped = rlp_for_group.groupby(["ZIP_CODE", "year", "month", "Fuel Type - Primary"], dropna = False).sum().reset_index().reset_index(drop=True)
rlp_grouped["avg_transaction_price"] = rlp_grouped.apply(lambda x: try_divide(x.TRANSACTION_PRICE, x.vehs_with_price), axis =1)

In [42]:
# Get evs
evs_grouped = rlp_grouped[rlp_grouped["Fuel Type - Primary"] == "Electric"]
evs_grouped["year"] = evs_grouped["year"].astype(int)

# Observe

In [ ]:
# Get counts
counts_df = evs_grouped.groupby(["year", "month", "ZIP_CODE"]).sum().reset_index().reset_index(drop=True)[["year", "month", "ZIP_CODE", "VEH_COUNT"]]
raw_counts = counts_df["VEH_COUNT"].to_list()
raw_counts_trunc = [x for x in raw_counts if x>1]

**Observe counts**

In [ ]:
counts,bins = np.histogram(raw_counts, bins = 20)
counts_trunc, bins_trunc = np.histogram(raw_counts_trunc, bins = 10)

In [ ]:
plt.stairs(counts[0:6],bins[0:7])

In [ ]:
print(f"We have clear overdispersion with E[counts]={np.mean(raw_counts):0.2f} and Var[counts]={np.var(raw_counts):0.2f}")

In [ ]:
plt.stairs(np.unique(np.array(raw_counts_trunc), return_counts = True)[1][0:47], np.unique(np.array(raw_counts_trunc), return_counts = True)[0])

In [ ]:
print(f"When truncated E[counts]={np.mean(raw_counts_trunc):0.2f} and Var[counts]={np.var(raw_counts_trunc):0.2f}")

# Attempt merge to percentages

In [127]:
evs_bg_merge = evs_grouped.merge(mun_evs_zipgroups[["Postal", "GEOID", "pct_within_zip"]],
                                 left_on = "ZIP_CODE",
                                 right_on = "Postal",
                                 how = "outer")

,ZIP_CODE,year,month,Fuel Type - Primary,VEH_COUNT,vehs_with_price,vehs_without_price,TRANSACTION_PRICE,avg_transaction_price,Postal,GEOID,pct_within_zip
0,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003460e+10,0.009259
1,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.003086
2,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.104938
3,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.064815
4,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.037037
...,...,...,...,...,...,...,...,...,...,...,...,...
793,6001.0,2022.0,12.0,Electric,16.0,13.0,3.0,743054.0,57158.0,6001.0,9.003462e+10,0.064815
794,6001.0,2022.0,12.0,Electric,16.0,13.0,3.0,743054.0,57158.0,6001.0,9.003462e+10,0.157407
795,6001.0,2022.0,12.0,Electric,16.0,13.0,3.0,743054.0,57158.0,6001.0,9.003462e+10,0.145062
796,6001.0,2022.0,12.0,Electric,16.0,13.0,3.0,743054.0,57158.0,6001.0,9.003466e+10,0.012346


In [136]:
evs_bg_merge[(evs_bg_merge["ZIP_CODE"]==6001) & (evs_bg_merge["year"]==2018) & (evs_bg_merge["month"]==1)]

,ZIP_CODE,year,month,Fuel Type - Primary,VEH_COUNT,vehs_with_price,vehs_without_price,TRANSACTION_PRICE,avg_transaction_price,Postal,GEOID,pct_within_zip
0,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003460e+10,0.009259
1,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.003086
2,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.104938
3,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.064815
4,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.037037
5,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.067901
6,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.169753
7,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.117284
8,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.043210
9,6001.0,2018.0,1.0,Electric,3.0,2.0,1.0,127479.0,63739.5,6001.0,9.003462e+10,0.064815


# Covariates prepare

In [14]:
income_data_path = data_path / "data_for_poisson" / "ACSS1901 - Income - By ZCTA"
population_data_path = data_path / "data_for_poisson" / "ACSB01003 - Total Population - By ZCTA"
zip_area_data_path = data_path / "data_for_poisson" / "zctas_2019_areas.csv"
education_data_path = data_path / "data_for_poisson" / "ACSS1501 - Educational Attainment - By ZCTA"

**Need to fix this**

In [15]:
def fix_income_df(input):
    out = input.replace(',', '')
    try:
        return int(out)
    except:
        return np.NaN

In [16]:
income_df = pd.DataFrame()

for file in income_data_path.iterdir():
    # Get the year
    year = file.name[7:11]

    # Read in the file and get the median income
    f = pd.read_csv(file, thousands = ',')
    f = f.iloc[[11]].T.reset_index()
    f = f.loc[f["index"].str.contains("Households!!Estimate")]

    # Get the zip code and year, and rename columns
    f["index"] = f["index"].str[6:11].astype(int)
    f["year"] = int(year)
    f = f.rename(columns = {"index" : "zip", 11: "median_income"}).reset_index(drop = True)
    f["median_income"] = f["median_income"].apply(lambda x: fix_income_df(x))

    # Append to the master df
    income_df = pd.concat([income_df, f]).reset_index(drop = True)

In [17]:
income_df = pd.concat([income_df, expand_df(income_df, 2021, 2022)])

In [18]:
population_df = pd.DataFrame()

for file in population_data_path.iterdir():
    # Get the year
    year = file.name[7:11]

    # Read in the file and get the population
    f = pd.read_csv(file, thousands = ',')
    f = f.T.reset_index()
    f = f.loc[f["index"].str.contains("Estimate")]

    # Get the zip code and year, and rename columns
    f["index"] = f["index"].str[6:11].astype(int)
    f["year"] = int(year)
    f = f.rename(columns = {"index" : "zip", 0: "population"}).reset_index(drop = True)
    f["population"] = f["population"].astype(int)

    # Append to the master df
    population_df = pd.concat([population_df, f]).reset_index(drop = True)

In [20]:
zip_areas = pd.read_csv(zip_area_data_path, index_col = [0]).reset_index(drop=True)[["ZCTA5CE10","Area_km2"]].rename(columns={"ZCTA5CE10":"zip"})
zip_areas = zip_areas.loc[zip_areas["zip"].astype(str).str[0:4].str.zfill(5).str[0:2]=="06"]
zip_areas["Area_km2"] = zip_areas["Area_km2"].astype(float)

In [21]:
population_df = population_df.merge(zip_areas,
                                    left_on = "zip",
                                    right_on = "zip",
                                    how = "left")

In [22]:
population_df

,zip,population,year,Area_km2
0,6001,18561,2017,61.577189
1,6002,20848,2017,67.930523
2,6010,60498,2017,69.416639
3,6013,9598,2017,78.783841
4,6016,7003,2017,39.016399
...,...,...,...,...
1411,6902,69523,2021,32.478960
1412,6903,14816,2021,47.280737
1413,6905,21604,2021,12.900674
1414,6906,10847,2021,3.131654


In [23]:
# Add density
population_df["pop_density"] = population_df.apply(lambda x: try_divide(x.population, x.Area_km2), axis = 1)

# Add in 2022
population_df = pd.concat([population_df, expand_df(population_df, 2021, 2022)])

In [25]:
education_data_df = pd.DataFrame()

In [26]:
for file in education_data_path.iterdir():
    # Get the year
    year = file.name[7:11]

    # Read in the file and get the population
    f = pd.read_csv(file, skiprows =[0])
    col1 = "Geographic Area Name"
    col2 = "Estimate!!Percent!!Population 25 years and over!!High school graduate (includes equivalency)"
    col2a = "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)"
    
    try:
        f = f[[col1, col2]]
    except:
        f = f[[col1, col2a]]

    f = f.rename(columns = {f.columns[0]:"zip", f.columns[1]:"pct_over25_hs"}).reset_index(drop=True)
    f["zip"] = f["zip"].str[6:].astype(int)
    f["year"] = int(year)
    education_data_df = pd.concat([education_data_df, f]).reset_index(drop=True)

In [27]:
education_data_df = pd.concat([education_data_df, expand_df(education_data_df, 2021, 2022)])

# Merge

In [48]:
evs_grouped_merge = evs_grouped.copy()

In [49]:
# Merge in population
evs_grouped_merge = evs_grouped_merge.merge(population_df,
                                            left_on = ["ZIP_CODE", "year"],
                                            right_on = ["zip","year"],
                                            how = "left")


In [50]:
# Add education data
evs_grouped_merge = evs_grouped_merge.merge(education_data_df,
                                            left_on = ["ZIP_CODE", "year"],
                                            right_on = ["zip","year"],
                                            how = "left")

In [51]:
# Add income data
evs_grouped_merge = evs_grouped_merge.merge(income_df,
                        left_on = ["ZIP_CODE", "year"],
                        right_on = ["zip","year"],
                        how = "left")

In [52]:
evs_grouped_merge = evs_grouped_merge.drop("zip_x", axis = 1)
evs_grouped_merge = evs_grouped_merge.drop("zip_y", axis = 1)
evs_grouped_merge = evs_grouped_merge.drop("zip", axis = 1)

In [87]:
check_na_mask = evs_grouped_merge["Area_km2"].isna()| evs_grouped_merge["pct_over25_hs"].isna() | evs_grouped_merge["median_income"].isna()
print(f"A total of {len(evs_grouped_merge[check_na_mask])} rows with missing data, or {check_na_mask.sum() / len(evs_grouped_merge):0.4f}%")

A total of 0 rows with missing data, or 0.0000%


In [85]:
evs_grouped_merge = evs_grouped_merge.dropna()

In [88]:
evs_grouped_merge.to_csv(data_path / "data_for_poisson"/ "poisson_data_112023.csv")

In [2]:
evs_grouped_merge = pd.read_csv("poisson_data_112023.csv", index_col = [0])

# Descriptives

## Dispersion analysis
Note: Not controlling for month, not controlling for before and after COVID

In [61]:
expecteds = evs_grouped_merge.groupby("ZIP_CODE")["VEH_COUNT"].mean()
vars = evs_grouped_merge.groupby("ZIP_CODE")["VEH_COUNT"].var()
dispersion = pd.DataFrame([expecteds, vars]).T.reset_index()
dispersion.columns= ["ZIP_CODE","expected", "variance"]
dispersion["difference"] = dispersion["expected"] - dispersion["variance"]

# Analysis
dispersion[abs(dispersion["difference"]) < (1*abs(dispersion["expected"]))]

,ZIP_CODE,expected,variance,difference
1,6002,2.700000,2.785714,-0.085714
3,6013,2.243902,2.189024,0.054878
4,6016,1.578947,0.812865,0.766082
5,6018,1.600000,1.155556,0.444444
6,6019,2.325000,2.532692,-0.207692
...,...,...,...,...
279,6896,3.660377,5.805515,-2.145138
281,6901,2.782609,2.351691,0.430918
284,6905,4.611111,8.883648,-4.272537
285,6906,2.250000,1.964286,0.285714


# Test Poisson

In [57]:
from sklearn.linear_model import PoissonRegressor

In [58]:
model = PoissonRegressor()

In [204]:
evs_grouped_merge

,ZIP_CODE,year,month,Fuel Type - Primary,VEH_COUNT,vehs_with_price,vehs_without_price,TRANSACTION_PRICE,avg_transaction_price,population,Area_km2,pct_over25_hs,median_income
0,6001,2018,1,Electric,3,2.0,1.0,127479.0,63739.50,18532.0,61.577189,10.1,"129,888"
1,6001,2018,3,Electric,6,2.0,4.0,58293.0,29146.50,18532.0,61.577189,10.1,"129,888"
2,6001,2018,4,Electric,7,6.0,1.0,212794.0,35465.67,18532.0,61.577189,10.1,"129,888"
3,6001,2018,5,Electric,1,1.0,0.0,28504.0,28504.00,18532.0,61.577189,10.1,"129,888"
4,6001,2018,6,Electric,4,2.0,2.0,54162.0,27081.00,18532.0,61.577189,10.1,"129,888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,6907,2022,8,Electric,3,3.0,0.0,164713.0,54904.33,8849.0,5.414747,20.7,"99,861"
8463,6907,2022,9,Electric,2,2.0,0.0,148069.0,74034.50,8849.0,5.414747,20.7,"99,861"
8464,6907,2022,10,Electric,4,3.0,1.0,139301.0,46433.67,8849.0,5.414747,20.7,"99,861"
8465,6907,2022,11,Electric,4,2.0,2.0,102634.0,51317.00,8849.0,5.414747,20.7,"99,861"


In [71]:
y = evs_grouped_merge.dropna()
y = y["VEH_COUNT"].to_numpy()
x = evs_grouped_merge.dropna()
x["population"] = x["population"]/1000
x["median_income"] = x["median_income"]/10000
x["pop_density"] = x["pop_density"]/100
x = x[["population", "pct_over25_hs", "median_income", "pop_density"]].to_numpy()

In [73]:
model.fit(x,y)

PoissonRegressor()

In [74]:
model.intercept_, model.coef_

(0.3609378626614327,
 array([ 0.03113661, -0.02208575,  0.07635376, -0.00382444]))

In [75]:
predicted = model.predict(x)

In [76]:
pd.DataFrame([y, predicted]).T

,0,1
0,3.0,5.447350
1,6.0,5.447350
2,7.0,5.447350
3,1.0,5.447350
4,4.0,5.447350
...,...,...
7638,3.0,2.409132
7639,2.0,2.409132
7640,4.0,2.409132
7641,4.0,2.409132


# Getting right columns

In [9]:
education_cols = []
for file in education_data_path.iterdir():
    f = pd.read_csv(file, skiprows = [0])
    cols = f.columns.tolist()
    a = [x for x in cols if "Percent" in x and "High school graduate" in x and not"Annotation" in x and not "Margin" in x and not "RACE" in x]
    a = [x for x in a if "25 years and over" in x]
    a = [x for x in a if not "Male" in x and not "Female" in x]
    a = [x for x in a if not "to" in x]
    education_cols.append([a])

In [10]:
education_cols

[[['Estimate!!Percent!!Population 25 years and over!!High school graduate (includes equivalency)',
   'Estimate!!Percent!!Population 25 years and over!!Population 65 years and over!!High school graduate or higher',
   'Estimate!!Percent!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency)']],
 [['Estimate!!Percent!!Population 25 years and over!!High school graduate (includes equivalency)',
   'Estimate!!Percent!!Population 25 years and over!!High school graduate or higher',
   'Estimate!!Percent!!Population 25 years and over!!Population 65 years and over!!High school graduate or higher',
   'Estimate!!Percent!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency)']],
 [['Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school gradua